In [1]:
%%writefile app.py
import streamlit as st
from img_classification import teachable_machine_classification
import keras
from PIL import Image, ImageOps
import numpy as np
st.title("Sistema de Diagnóstico asistido por ordenador")
st.header("Clasificación de cáncer de piel mediante CNN")
st.text("Por favor introduce una imagen de una lesión cutánea que pueda sospechar DFSP, carcinoma o melanoma:")

uploaded_file = st.file_uploader("Escoge una lesión cutánea ...", type="jpg")
if uploaded_file is not None:
  image = Image.open(uploaded_file)
  st.image(image, caption='Imagen subida exitosamente.', use_column_width=True)
  st.write("")
  st.write("Clasificando...")
  label = teachable_machine_classification(image, 'train/model.h5') # Name of the model from Teachablemachine
  if label == 0:
    st.write("The MRI scan has a brain tumor")
  else:
    st.write("The MRI scan is healthy")

Overwriting app.py


In [2]:
%%writefile img_classification.py
import tensorflow.keras
from PIL import Image, ImageOps
import numpy as np


def teachable_machine_classification(img, weights_file):
    # Load the model
    model = tensorflow.keras.models.load_model(weights_file)

    # Create the array of the right shape to feed into the keras model
    data = np.ndarray(shape=(1, 224, 224, 3), dtype=np.float32)
    image = img
    #image sizing
    size = (224, 224)
    image = ImageOps.fit(image, size, Image.ANTIALIAS)

    #turn the image into a numpy array
    image_array = np.asarray(image)
    # Normalize the image
    normalized_image_array = (image_array.astype(np.float32) / 127.0) - 1

    # Load the image into the array
    data[0] = normalized_image_array  # (Not sure if this is needed, but gives an error!!!)

    # run the inference
    prediction = model.predict(data)
    return np.argmax(prediction) # return position of the highest probability

Overwriting img_classification.py


In [3]:
from pyngrok import ngrok
public_url = ngrok.connect(port='80')
print('Link to web app:')
print (public_url)
!streamlit run --server.port 80 app.py >/dev/null

Link to web app:
NgrokTunnel: "http://dd7b2b459f45.ngrok.io" -> "http://localhost:80"
2021-06-03 05:23:50.443 An update to the [server] config option section was detected. To have these changes be reflected, please restart streamlit.
Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/bin/streamlit", line 8, in <module>
    sys.exit(main())
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 829, in __call__
    return self.main(*args, **kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 782, in main
    rv = self.invoke(ctx)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 1259, in invoke
    return _process_result(sub_ctx.command.invoke(sub_ctx))
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/click/core.py", line 1066, in invoke
    return ctx.invoke(self.

t=2021-06-03T05:24:17+0000 lvl=warn msg="failed to open private leg" id=7ff875a7e107 privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
t=2021-06-03T05:24:17+0000 lvl=warn msg="failed to open private leg" id=ed887e0d6aeb privaddr=localhost:80 err="dial tcp 127.0.0.1:80: connect: connection refused"
